1. S --> Variable=Expr 
2. Expr --> Sum
3. Sum --> ProductSum'
4. Sum' --> [+|-]Sum | $\varepsilon$
5. Product --> OrProduct'
6. Product' --> [*|/]Product | $\varepsilon$
7. Or --> XorOr'
8. Or' --> "|"Or | $\varepsilon$
9. Xor --> AndmXor'
10. Xor' --> ^Xor | $\varepsilon$
11. And --> TermAnd'
12. And' --> &And | $\varepsilon$
13. Term --> InvTerm
14. Inv --> ! | $\varepsilon$
15. Term --> (Expr) | Number | Variable | Bool

* Number: \d+ (В этой библиотеке во время парсинга применяется int())
* Variable: [A-Za-z]
* Bool: True|False

1. Корректным названием переменной является большая или маленькая латинская буква.
2. Перезаписывать переменные разрешается
3. Тип используемых переменных - стандартный Python тип

In [1]:
from pyparsing import infixNotation, opAssoc, Keyword, Word, alphas, ParserElement, oneOf, pyparsing_common
from classes import *

EPS = 1e-5

ParserElement.enablePackrat()

TRUE = Keyword("True")
FALSE = Keyword("False")
integer = pyparsing_common.integer # helps to parse unsigned int
variable = Word(alphas, exact=1) # helps to parse variable (should consist)

operand = TRUE | FALSE | integer | variable
operand.setParseAction(Operand)

# define expression, based on expression operand and
# list of operations in precedence order

operators = [
    ("!", 1, opAssoc.RIGHT, BoolNot),
    ("&", 2, opAssoc.LEFT, BoolAnd),
    ("^", 2, opAssoc.LEFT, BoolXOR),
    ("|", 2, opAssoc.LEFT, BoolOr),
    ("/", 2, opAssoc.LEFT, IntDiv),
    ("*", 2, opAssoc.LEFT, IntMult),
    ("-", 2, opAssoc.LEFT, IntMinus),        
    ("+", 2, opAssoc.LEFT, IntPlus),
]

expr = variable + '=' + infixNotation(operand, operators)

variables_dict = {}

tests = []
with open('generated_bool_tests.txt')  as f:
    lines = f.readlines()
    for line in lines:
        sp_l = line.split(',')
        tests.append((sp_l[0], eval(sp_l[1])))

cnt_fail = 0
cnt_err = 0
cnt_correct = 0
for t, expected in tests:
    try:
        var, _, res = expr.parseString(t, parseAll=True)
        success = "PASS" if expected - EPS< res.calc() < expected + EPS else "FAIL"
        variables_dict[var] = res.calc()
        if success == "FAIL":
            cnt_fail += 1
            print(success, "\ntest:", t, "\nparsed: ", res, "=", res.calc(), "\ncorrect =", expected)
        else:
            cnt_correct += 1
    except BaseException as e:
        cnt_err += 1
        print("ERROR", e)
        print("test:", t)
        print("_" * 50)
        
print('Всего тестов: {}, из них: \n\tправильных: {}, \
\n\tс ошибкой при вычислении: {}, \n\tс ошибкой при парсинге: {}' \
      .format(len(tests), cnt_correct, cnt_fail, cnt_err))

Всего тестов: 768, из них: 
	правильных: 768, 
	с ошибкой при вычислении: 0, 
	с ошибкой при парсинге: 0
